In [69]:
import pandas as pd

In [70]:
df = pd.read_csv("FAOSTAT.csv")

df.head()

,Column1,Year,Item,continents,Area,Flag Description,Unit,Value
0,0,2000,Average dietary energy requirement (kcal/cap/day),Africa,Algeria,Estimated value,kcal/cap/d,2308.0
1,1,2000,Average dietary energy requirement (kcal/cap/day),Africa,Angola,Estimated value,kcal/cap/d,2121.0
2,2,2000,Average dietary energy requirement (kcal/cap/day),Africa,Benin,Estimated value,kcal/cap/d,2165.0
3,3,2000,Average dietary energy requirement (kcal/cap/day),Africa,Botswana,Estimated value,kcal/cap/d,2286.0
4,4,2000,Average dietary energy requirement (kcal/cap/day),Africa,Burkina Faso,Estimated value,kcal/cap/d,2183.0


In [71]:
df['Flag Description'].unique()

array(['Estimated value', 'Figure from international organizations',
       'Missing value', 'Missing value; suppressed', 'Official figure'],
      dtype=object)

In [72]:
# Drop rows where flag description is "Missing Value", "Missing value; suppressed"
df['Flag Description'].unique()

df = df[~df['Flag Description'].isin(["Missing value", "Missing value; suppressed"])]

In [73]:
pivot_df = df.pivot_table(index = ['Year','continents','Area','Flag Description'], columns = 'Item',values= 'Value').reset_index()

In [74]:
# Expand interval years for richer dataset

## Input is a series
def expand_year(row):
    y = row["Year"]
    if isinstance(y, str) and "-" in y:
        start, end = map(int, y.split("-"))
        expanded = []
        for yr in range(start, end + 1):
            row_copy = row.copy()
            row_copy["Year"] = str(yr)
            expanded.append(row_copy)
        return expanded
    else:
        return [row]
    
   

In [75]:
for _, row in df.iterrows():
    print(row)
    break

Column1                                                             0
Year                                                             2000
Item                Average dietary energy requirement (kcal/cap/day)
continents                                                     Africa
Area                                                          Algeria
Flag Description                                      Estimated value
Unit                                                       kcal/cap/d
Value                                                          2308.0
Name: 0, dtype: object


In [76]:
# Expand rows
expanded_rows = []
for _, row in pivot_df.iterrows():
    expanded_rows.extend(expand_year(row))

# Recreate the DataFrame
df_expanded = pd.DataFrame(expanded_rows)

In [77]:
# Define columns that are used to identify unique rows
key_cols = ['Year', 'continents', 'Area','Flag Description']

# Merge rows by taking first non-null for each group
df_merged = (
    df_expanded
    .groupby(key_cols, as_index=False)
    .first()
)

In [78]:
# df_merged.to_csv("Pivotted_considering_flag.csv",index=False)

## Break

In [79]:
conflict_columns = {}

# Check for conflicts
value_columns = [col for col in df_merged.columns if col not in ['Year', 'Area','continents', 'Flag Description']]

for col in value_columns:
    sub_df = df_merged[[col, 'Flag Description']].dropna(subset=[col])
    flags_present = sub_df['Flag Description'].unique()
    
    if len(flags_present) > 1:
        conflict_columns[col] = flags_present.tolist()

In [80]:
# conflict_columns

In [81]:
conflicted_cols = list(conflict_columns.keys())
non_conflicted_cols = [col for col in value_columns if col not in conflicted_cols]

# Define flag priority
flag_priority = {
    'Official figure': 0,
    'Estimated value': 1,
    'Figure from international organizations': 2 # Not really needed because it does not conflict with other flags
}

# Assign flag rank
df_merged['Flag Rank'] = df_merged['Flag Description'].map(flag_priority)

# Keep only necessary columns for conflicted + rank
df_conflicted = df_merged[['Year', 'continents', 'Area', 'Flag Rank'] + conflicted_cols].copy()
# df_conflicted = df_conflicted.sort_values(by=['Year', 'continents', 'Area', 'Flag Rank'])
# df_conflicted_cleaned = df_conflicted.drop_duplicates(subset=['Year', 'continents', 'Area'], keep='first')
df_conflicted_cleaned = (
    df_conflicted
    .sort_values(by=['Year', 'continents', 'Area', 'Flag Rank'])
    .groupby(['Year', 'continents', 'Area'], as_index=False)
    .first()
)

# Handle non-conflicted columns using .first()
df_non_conflicted = (
    df_merged[['Year', 'continents', 'Area'] + non_conflicted_cols]
    .groupby(['Year', 'continents', 'Area'], as_index=False)
    .first()
)

# Final merge
df_final = pd.merge(df_non_conflicted, df_conflicted_cleaned, on=['Year', 'continents', 'Area'], how='outer')
df_final = df_final.drop(columns=['Flag Rank'])


group_keys = ['Year', 'continents', 'Area']
other_cols = sorted([col for col in df_final.columns if col not in group_keys])

df_final1 = df_final[group_keys + other_cols]


In [82]:
df_final1.to_csv("Pivotted_final_updated.csv")

In [83]:
df_final1.head()

Item,Year,continents,Area,Average dietary energy requirement (kcal/cap/day),Average dietary energy supply adequacy (percent) (3-year average),Average fat supply (g/cap/day) (3-year average),Average protein supply (g/cap/day) (3-year average),Average supply of protein of animal origin (g/cap/day) (3-year average),Cereal import dependency ratio (percent) (3-year average),Coefficient of variation of habitual caloric consumption distribution (real number),...,Prevalence of moderate or severe food insecurity in the male adult population (percent) (3-year average),Prevalence of moderate or severe food insecurity in the total population (percent) (3-year average),Prevalence of obesity in the adult population (18 years and older) (percent),Prevalence of severe food insecurity in the female adult population (percent) (3-year average),Prevalence of severe food insecurity in the male adult population (percent) (3-year average),Prevalence of severe food insecurity in the total population (percent) (3-year average),Prevalence of undernourishment (percent) (3-year average),Rail lines density (total route in km per 100 square km of land area),"Share of dietary energy supply derived from cereals, roots and tubers (percent) (3-year average)",Value of food imports in total merchandise exports (percent) (3-year average)
0,2000,Africa,Algeria,2308.0,126.0,67.0,77.7,19.3,79.7,0.30,...,NaN,NaN,13.4,NaN,NaN,NaN,7.5,0.2,59.0,13.0
1,2000,Africa,Angola,2121.0,74.0,39.4,40.9,12.8,57.1,0.30,...,NaN,NaN,5.3,NaN,NaN,NaN,67.4,NaN,66.0,7.0
2,2000,Africa,Benin,2165.0,108.0,48.5,56.2,9.5,13.0,0.28,...,NaN,NaN,4.5,NaN,NaN,NaN,17.2,NaN,72.0,59.0
3,2000,Africa,Botswana,2286.0,98.0,48.4,66.5,25.6,90.9,0.24,...,NaN,NaN,10.0,NaN,NaN,NaN,23.7,NaN,50.0,12.0
4,2000,Africa,Burkina Faso,2183.0,108.0,53.2,69.6,9.7,9.0,0.34,...,NaN,NaN,1.4,NaN,NaN,NaN,22.7,0.2,65.0,52.0


## Standardize country and country code (Please ignore)

In [84]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [85]:
url = "https://www.countrycode.org/"
response = requests.get(url)

In [86]:
soup = BeautifulSoup(response.content,"html.parser")

In [87]:
table = soup.find("table",class_="main-table")

In [88]:
headers = [header.text.strip() for header in table.find_all("th")]
headers

['COUNTRY', 'COUNTRY CODE', 'ISO CODES', 'POPULATION', 'AREA KM2', 'GDP $USD']

In [89]:
rows = []
for tr in table.find_all("tr")[1:]:
    cells = tr.find_all("td")
    row = [cell.text.strip() for cell in cells]
    if row:
        rows.append(row)

In [90]:
rows

[['Afghanistan', '93', 'AF / AFG', '29,121,286', '647,500', '20.65 Billion'],
 ['Albania', '355', 'AL / ALB', '2,986,952', '28,748', '12.8 Billion'],
 ['Algeria', '213', 'DZ / DZA', '34,586,184', '2,381,740', '215.7 Billion'],
 ['American Samoa', '1-684', 'AS / ASM', '57,881', '199', '462.2 Million'],
 ['Andorra', '376', 'AD / AND', '84,000', '468', '4.8 Billion'],
 ['Angola', '244', 'AO / AGO', '13,068,161', '1,246,700', '124 Billion'],
 ['Anguilla', '1-264', 'AI / AIA', '13,254', '102', '175.4 Million'],
 ['Antarctica', '672', 'AQ / ATA', '0', '14,000,000', ''],
 ['Antigua and Barbuda', '1-268', 'AG / ATG', '86,754', '443', '1.22 Billion'],
 ['Argentina', '54', 'AR / ARG', '41,343,201', '2,766,890', '484.6 Billion'],
 ['Armenia', '374', 'AM / ARM', '2,968,000', '29,800', '10.44 Billion'],
 ['Aruba', '297', 'AW / ABW', '71,566', '193', '2.516 Billion'],
 ['Australia', '61', 'AU / AUS', '21,515,754', '7,686,850', '1.488 Trillion'],
 ['Austria', '43', 'AT / AUT', '8,205,000', '83,858', 

In [91]:
df_country = pd.DataFrame(rows,columns = headers)

In [92]:
df_country = df_country[["COUNTRY","ISO CODES"]]

In [93]:
df_country[['ISO1', 'ISO2']] = df_country['ISO CODES'].str.split("/", expand=True)

In [94]:
df_country = df_country[["COUNTRY","ISO1","ISO2"]]
df_country

,COUNTRY,ISO1,ISO2
0,Afghanistan,AF,AFG
1,Albania,AL,ALB
2,Algeria,DZ,DZA
3,American Samoa,AS,ASM
4,Andorra,AD,AND
...,...,...,...
235,Wallis and Futuna,WF,WLF
236,Western Sahara,EH,ESH
237,Yemen,YE,YEM
238,Zambia,ZM,ZMB


In [57]:
areas_in_final1 = set(df_final1["Area"].unique())
countries_in_country_df = set(df_country["COUNTRY"].unique())

only_in_final1 = areas_in_final1 - countries_in_country_df
print("In df_final1 but not in df_country:", only_in_final1)

only_in_df_country = countries_in_country_df - areas_in_final1
print("In df_country but not in df_final1:", only_in_df_country)

In df_final1 but not in df_country: {'North Macedonia', 'Cabo Verde', 'Eswatini', "Côte d'Ivoire", 'Timor-Leste'}
In df_country but not in df_final1: {'Saint Helena', 'Hong Kong', 'Tanzania', 'Puerto Rico', 'Gibraltar', 'Vatican', 'Russia', 'French Polynesia', 'Northern Mariana Islands', 'South Korea', 'Myanmar', 'Tonga', 'British Indian Ocean Territory', 'Ivory Coast', 'Macedonia', 'Palau', 'Aruba', 'Australia', 'Sint Maarten', 'Venezuela', 'Curacao', 'Antarctica', 'Sao Tome and Principe', 'Vanuatu', 'Guernsey', 'Micronesia', 'Fiji', 'Svalbard and Jan Mayen', 'Cape Verde', 'Taiwan', 'British Virgin Islands', 'Syria', 'Solomon Islands', 'U.S. Virgin Islands', 'Jersey', 'Monaco', 'Saint Martin', 'Saint Barthelemy', 'Iran', 'Bolivia', 'Falkland Islands', 'United Kingdom', 'American Samoa', 'Mayotte', 'Cook Islands', 'Saint Pierre and Miquelon', 'Christmas Island', 'Vietnam', 'East Timor', 'Cayman Islands', 'Moldova', 'San Marino', 'Nauru', 'Netherlands', 'Liechtenstein', 'Kosovo', 'Angui

In [95]:
df_with_iso = pd.merge(df_final1,df_country, left_on= 'Area',right_on= 'COUNTRY',how = "left")

In [96]:
df_with_iso.to_csv("Test.csv")